In [1]:
import csv
import psycopg2
from psycopg2 import sql

In [6]:
data_path = '../../../data/raw/l_amat_20240501/AM.dat'

In [2]:
db_params = {
    'dbname': 'ross_projects',
    'user': 'rwardrup',
    'password': 'Rward0232',
    'host': '192.168.3.231'
}

In [10]:
create_table_query = """
CREATE TABLE if not exists ham_ops.amateur_data (
    record_type CHAR(2),
    unique_system_identifier NUMERIC(9,0),
    uls_file_number CHAR(14),
    ebf_number VARCHAR(30),
    call_sign CHAR(10),
    operator_class CHAR(1),
    group_code CHAR(1),
    region_code SMALLINT,
    trustee_call_sign CHAR(10),
    trustee_indicator CHAR(1),
    physician_certification CHAR(1),
    ve_signature CHAR(1),
    systematic_call_sign_change CHAR(1),
    vanity_call_sign_change CHAR(1),
    vanity_relationship VARCHAR(12),
    previous_call_sign CHAR(10),
    previous_operator_class CHAR(1),
    trustee_name VARCHAR(50)
);"""

In [11]:
def setup_database(params, query):
    conn = psycopg2.connect(**params)
    cursor = conn.cursor()
    cursor.execute(query)
    conn.commit()
    cursor.close()
    conn.close()

# Function to insert data into the database
def insert_data(params, data):
    conn = psycopg2.connect(**params)
    cursor = conn.cursor()
    insert_query = sql.SQL("""
    INSERT INTO ham_ops.amateur_data VALUES ({})
    """).format(sql.SQL(', ').join(sql.Placeholder() * len(data[0])))
    cursor.executemany(insert_query, data)
    conn.commit()
    cursor.close()
    conn.close()

# Function to process the .dat file and load data
def process_and_load(filename, db_params):
    with open(filename, 'r', newline='') as file:
        reader = csv.reader(file, delimiter='|')
        data = [[None if cell == '' else cell for cell in row] for row in reader]
    insert_data(db_params, data)

In [12]:
# Set up the database table
setup_database(db_params, create_table_query)

# Process the .dat file and load the data into the database
process_and_load(data_path, db_params)